In [6]:
%matplotlib inline
from astropy.io import fits
from astropy import coordinates as c
import matplotlib.pyplot as plt
import numpy as np
import sqlite3 as sql
import barycorrpy
import csv

In [7]:
data_dir = 'data/radial_velocity/'

rv_data = {"Anderson_2010_WASP17.csv": {"time": "BJD", "time_offset": 2450000,
                "target": "Wasp-17", "instrument": "multi",},
           "Bayliss_2010_WASP17.csv": {"time": "HJD", "time_offset": 0,
                "columns": ["","",""]},
           "apj493234t3_mrt.txt": {"time": "BJDTDB", "time_offset": 0, 
                "instrument": "HIRES",
                "columns": ["time","rv","rv_err", "target"]},
           "J_A+A_524_A25/rvdata.dat": {"time": "BJD", "time_offset": 0,
                "columns": ["target","instrument","time","rv","rv_err"]}
          }

In [11]:
# Do the simplest one first

conn = sql.connect('shel_database.sqlite')
cur = conn.cursor()

with open(data_dir + 'apj493234t3_mrt.txt', 'r') as f:
    freader = csv.reader(f, delimiter = ' ')
    for row in freader:
        if row[0][0] == "#":
            continue
        # deal with variable whitespace
        data = [x for x in row if x != ""]
        target_id = cur.execute(f'select id from targets where name = "{data[3]}"').fetchall()
        if target_id == []:
            continue
        target_id = target_id[0][0]
        # This is data ref 1, need to code generalized ref_id retrieval
        stmt = ("insert into radial_velocities (target_id, reference_id, "
                f"instrument, bjd, rv, rv_err) values ({target_id}, 1, 5, {data[0]},"
                f"{data[1]}, {data[2]});")
        cur.execute(stmt)
        
conn.commit()
cur.close()
conn.close()